In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def vicsek_step(r, theta, L, v0, R, eta, rng):
    """
    r     : (N,2) positions in [0,L)
    theta : (N,)  angles in [-pi, pi)
    """
    N = r.shape[0]

    # 近傍探索
    # 周期境界での最短距離（minimum image convention）
    dr = r[:, None, :] - r[None, :, :]           # (N,N,2)
    dr = dr - L * np.round(dr / L)               # wrap to [-L/2, L/2)
    dist2 = np.sum(dr**2, axis=2)                # (N,N)
    neighbors = dist2 <= R**2                    # (N,N) bool (自分自身も含む)

    # 向き更新：theta_{j}^{t+1} = arg( sum_{k~j} exp(i theta_k^t) ) + noise
    exp_i = np.exp(1j * theta)                   # (N,)
    # 近傍ごとの複素和： (N,N) @ (N,) -> (N,)
    local_sum = neighbors @ exp_i
    mean_angle = np.angle(local_sum)

    noise = rng.uniform(-eta/2, eta/2, size=N)
    theta_new = mean_angle + noise
    # [-pi, pi) に正規化（必須ではないけど見通し良くなる）
    theta_new = (theta_new + np.pi) % (2*np.pi) - np.pi

    # 位置更新：r^{t+1} = r^t + v0 * e_{theta^{t+1}}
    step_vec = v0 * np.column_stack((np.cos(theta_new), np.sin(theta_new)))
    r_new = (r + step_vec) % L                   # 周期境界

    return r_new, theta_new

def order_parameter(theta):
    """P(t) = | (1/N) sum exp(i theta_j(t)) |"""
    return np.abs(np.mean(np.exp(1j * theta)))

In [ ]:

N = 500         # 粒子数
L = 10.0        # 系の一辺（密度 ~ N/L^2）
v0 = 0.03       # 1ステップの移動距離
R = 1.0         # 相互作用半径
eta = 0.3       # ノイズ強度
T = 500         # ステップ数
seed = 0

rng = np.random.default_rng(seed)

# 初期条件：位置一様、向きランダム
r = rng.uniform(0, L, size=(N, 2))
theta = rng.uniform(-np.pi, np.pi, size=N)

P_list = []

for t in range(T):
    # 観測（t時点）
    P_list.append(order_parameter(theta))
    # 1ステップ更新
    r, theta = vicsek_step(r, theta, L, v0, R, eta, rng)

# プロット
plt.figure()
plt.plot(P_list)
plt.xlabel("t")
plt.ylabel("P(t)")
plt.show()

import pandas as pd

# 時刻 t と P(t) をまとめる
df = pd.DataFrame({
    "t": np.arange(len(P_list)),
    "P": P_list
})

# CSVに保存
csv_name = "vicsek_P_t.csv"
df.to_csv(csv_name, index=False)

print(f"{csv_name} を保存しました")

In [ ]:
 from google.colab import files
 files.download("vicsek_P_t.csv")

In [ ]:
# import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# アニメーション用パラメータ
N = 400
L = 10.0
v0 = 0.05
R = 1.0
eta = 0.25
seed = 0

rng = np.random.default_rng(seed)
r = rng.uniform(0, L, size=(N, 2))
theta = rng.uniform(-np.pi, np.pi, size=N)

# 描画準備
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_aspect("equal")
ax.set_title("Vicsek model")

# quiver: 矢印で向きを表示（位置 r と方向 theta を描く）
U = np.cos(theta)
V = np.sin(theta)
q = ax.quiver(r[:,0], r[:,1], U, V, angles="xy", scale_units="xy", scale=3)

# 上に整列度Pを表示
text = ax.text(0.02, 1.02, "", transform=ax.transAxes)

# 1フレーム更新関数
def update(frame):
    global r, theta
    # 1ステップ進める
    r, theta = vicsek_step(r, theta, L, v0, R, eta, rng)

    # 描画データ更新
    U = np.cos(theta)
    V = np.sin(theta)
    q.set_offsets(r)
    q.set_UVC(U, V)

    # 整列度表示
    P = order_parameter(theta)
    text.set_text(f"t={frame:4d}   P={P:.3f}   eta={eta}")

    return q, text

# アニメ生成
frames = 300      # 総フレーム数
interval = 30     # ms
anim = FuncAnimation(fig, update, frames=frames, interval=interval, blit=False)

plt.close(fig)  # 静止画が重複表示されるのを防ぐ（Colab用）
HTML(anim.to_jshtml())

In [ ]:
import numpy as np
import pandas as pd

# 実験条件
N = 500
L = 10.0
v0 = 0.03
R = 1.0
T = 800
burn_in = 300
seed = 0

etas = np.linspace(0.0, 1.0, 11)  # ノイズ強度
rng = np.random.default_rng(seed)

def run_vicsek_mean_P(eta):
    r = rng.uniform(0, L, size=(N, 2))
    theta = rng.uniform(-np.pi, np.pi, size=N)

    P_vals = []
    for t in range(T):
        if t >= burn_in:
            P_vals.append(order_parameter(theta))
        r, theta = vicsek_step(r, theta, L, v0, R, eta, rng)

    return np.mean(P_vals)

# ηごとの ⟨P⟩ を計算
P_means = []
for eta in etas:
    P_means.append(run_vicsek_mean_P(float(eta)))

# CSVに保存
df = pd.DataFrame({
    "eta": etas,
    "P_mean": P_means
})

csv_name = "vicsek_eta_Pmean.csv"
df.to_csv(csv_name, index=False)

print(f"{csv_name} を保存しました")

In [ ]:
from google.colab import files
files.download("vicsek_eta_Pmean.csv")